In [17]:
from keras.layers import Input,Dense,Reshape,Flatten,BatchNormalization,LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

In [2]:
# Defining image dimensions

imgRows = 28
imgCols = 28
channels = 1
imgShape = (imgRows,imgCols,channels)

In [5]:
def generator():

    """
        Creates a generator model for a Generative Adversaial Network (GAN).

        The generator takes a noise vector as input and transforms it into realistic looking image through a series of fully connected layers,Leaky Relu 
        activation , batch normalizations and reshaping. The final output is scaled tot the range [-1,1] using the 'tanh' activation function,
        making it suitable for image generation tasks.
    
        return:
            keras.Model : A keras model that maps noise vectors to generated images.
    
    """

    #Define the shape of the noise vector ; this will serve as the input to the generastor 
    # typically used in GANS, the noise vector allows the model to generate diverse outputs
    noise_shape = (100,)

    # Initilize a sequential model , which is a linear stack of layers
    model = Sequential()

    #Add a dense layer with 256 neurons, this layer acts as the first fully connected layer, transforming the 
    #input noise vector into a higher dimensional feature space.
    #the input_shape defines the expected input noise shape dimensions 
    model.add(Dense(256,input_shape = noise_shape))
    
    # Add a LekyRelu activation function with a small negative slope defined by 'alpha'
    # LeakyRelu is a variant of the standard Relu (Rectified linear unit) actibvtion function
    # While standard RELU sets the value to 0 for all the negative inputs , this allows a small, non zero gradient for negative inputs
    # This helps mitigate the "dying RELU" problem, where neurons become inactive and stop learning due to 0 gradient
    # the alpha parameter defines the slope of the activatioin function for negative inputs. A smaller alpha means means less contibution from negative values 
    # large alpha means it allows more contribution from neagtive values.
    # this activation helps prevent the 'dying relu' problem by allowing small gradient for negative inputs
    model.add(LeakyReLU(alpha = 0.2))

    # Add batch normalization layer to stablize and accelarate training by normalizing the activations of the previous layer.
    # The momentum parameter controls how much of the past running statics to use.
    # This layer also prevents internal covariate shift and improves the models generalization ability
    model.add(BatchNormalization(momentum = 0.8))

    # Add a Dense layer with 512 neurons to further expand the feature space.
    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))

    # Add another Dense layer with 1024 neurons for further expansion.
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))

    # Add the output Dense Layer to produce the final generated image.
    # np.prod(image shape) caluclates the total number of pixels (flattned shape) of the output image.
    # the activation function 'tanh' ensures that the output values are scaled bwteen -1 and 1 which is common for image generation tasks 
    model.add(Dense(np.prod(imgShape),activation = 'tanh'))

    # Reshape the output to match the desired image dimensions(img Shape).
    model.add(Reshape(imgShape))

    #print model summary of the model architecture.
    model.summary()

    # define the input to the generastor which is the noise vector
    noise = Input(shape = noise_shape)

    # pass the noise vector through the model to generate image 
    img = model(noise)

    # return the generator model which maps noise vector to generated images
    return Model(noise, img)

In [6]:
generator()

/mnt/c/Users/somes/gans/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 256)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 784)            │       803,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 28, 28, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,493,520 (5.70 MB)

 Trainable params: 1,489,936 (5.68 MB)

 Non-trainable params: 3,584 (14.00 KB)

<Functional name=functional_12, built=True>

In [13]:
def discriminator():
    """
    Builds a desciminator model for Generative adversial network (GAN)

    The descriminator acts as a binary classifier that distuingishes between real and fake images.
    It takes an image as input ,flatten it into vector, and processes it through series of fully connected layers with leakyRelu activations. 
    The final layer uses sigmoid activation function output the porbality value representing the validity of the input image.

    Returns:
    Keras.Model : A keras model that maps an input image to validity score(0  to 1)
    
    """

    # initilize a sequential model for descriminators
    model = Sequential()

    # Flatten the input image from its original shape in 1D vector
    # this prepares the image for fully connected layers
    model.add(Flatten(input_shape = imgShape))

    #Add a dense layer with 512 neurons to process the flatten layer
    # this layer helps in learning higher-level features from input
    model.add(Dense(512))

    # add leaky relu activation function to introduce non-linearity.
    #the small negative slope (alpha = 0.2) ensures small gradient for negative inputs.
    model.add(LeakyReLU(alpha=0.2))

    #add another Dense Layer with 256 neurons for further feature extraction
    model.add(Dense(256))

    #add another LeakyRelu activation for non - linearity.
    model.add(LeakyReLU(alpha=0.2))

    # add the output Dense layer with 1 neuron and a single sigmoid activation function.
    #Thje sigmoid function outputs probality between 0 and 1 , representing wheather the input image is real (closer to 1 ) or fake (closer to 0).
    
    model.add(Dense(1,activation = 'sigmoid'))

    # prints the model summary
    model.summary()

    # defines the input to the descriminator, which is an image with same shape as the generator shape
    img = Input(shape = imgShape)

    #pass the input through the model to get the validity score.
    validity = model(img)

    #return the descriminator model, which maps an input to a validity score.
    return Model(img,validity)
    
    

In [14]:
discriminator()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 533,505 (2.04 MB)

 Trainable params: 533,505 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

<Functional name=functional_21, built=True>

In [15]:
def train( eopchs,batchSize = 128, saveInterval = 50):
    
    # load the msist datset
    ds_train, ds_info = tfds.load( 'mnist',split='train',shuffle_files=True,as_supervised=True, with_info=True)
    ds_train_images = ds_train.map(lambda image, label: image)
    def normalize_img(image):
          # Normalize the image: uint8 to float32 in range [-1, 1]
        image = tf.cast(image, tf.float32) / 127.5 - 1.0
        
        # Expand dimensions to add a channel axis (e.g., [28, 28] -> [28, 28, 1])
        image = tf.expand_dims(image, axis=-1)
        
        return image
    
    ds_train_images = ds_train_images.map(
        normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train_images = ds_train_images.cache()
    ds_train_images = ds_train_images.batch(batchSize)
    ds_train_images = ds_train_images.prefetch(tf.data.AUTOTUNE)

    halfBatch = int(batchSize/2)


    for epoch in  range(epochs):

        
    
    

dtype('uint8')

In [ ]:
"asdasdasd"
